In [10]:
import numpy as np
from ENDFtk.tree import Tape
from ENDFtk.MF2.MT151 import (ParticlePairs, ResonanceChannels, ResonanceParameters, SpinGroup, RMatrixLimited, ResonanceRange, Isotope, Section)

tape = Tape.from_file('/home/sole-pie01/ndlib/endfb8-neutron/n-029_Cu_063.endf')# OK

MATMF2MT151 = tape.MAT(tape.material_numbers[0]).MF(2).MT(151).parse()

isotopeIndex = 0 # Always zero
rangeIndex = 0 # User defined
RANGE = MATMF2MT151.isotopes[isotopeIndex].resonance_ranges[rangeIndex]
pairs = ParticlePairs( RANGE.parameters.particle_pairs )

groups = []
for sgroup in RANGE.parameters.spin_groups.to_list():
    groups.append( SpinGroup( ResonanceChannels( sgroup.channels ),
                              ResonanceParameters( sgroup.parameters.ER, sgroup.parameters.GAM ) ))
    

parameters = RMatrixLimited( ifg = RANGE.parameters.IFG, 
                             krl = RANGE.parameters.KRL, 
                             krm = RANGE.parameters.KRM,
                             pairs = pairs, 
                             groups = groups            )

isotopes = [ Isotope( zai = MATMF2MT151.isotopes[isotopeIndex].ZAI, 
                      abn = MATMF2MT151.isotopes[isotopeIndex].ABN, 
                      lfw = MATMF2MT151.isotopes[isotopeIndex].LFW,
                      ranges = [ ResonanceRange( el = RANGE.EL, 
                                                 eh = RANGE.EH,
                                                 naps = RANGE.NAPS, 
                                                 parameters = parameters ) ] ) ]
dummySection = Section( zaid = MATMF2MT151.ZA, awr = MATMF2MT151.AWR, isotopes = isotopes )

# tape.MAT(tape.material_numbers[0]).MF(2).insert_or_replace( dummySection )
# tape.to_file("toto.endf")


In [3]:
import ENDFtk
import NDSampler
from NDSampler.resonance.RMatrixLimited.Parameters_RML_RRR import RMatrixLimited

# StoreRMatrixLimited = RMatrixLimited()
# StoreRMatrixLimited.from_endftk(range = RANGE)

# Test assignment
# StoreRMatrixLimited.ListSpinGroup[0].ResonanceParameters.ER[1] = -200

MATMF32MT151 = tape.MAT(tape.material_numbers[0]).MF(32).MT(151).parse()

assert RMatrixLimited().from_endftk(mf2_range = RANGE, mf32_range = MATMF32MT151.isotopes[0].resonance_ranges[0]).reconstruct().to_string(2925, 2, 151) == parameters.to_string(2925, 2, 151)

In [12]:
import ENDFtk
import NDSampler
from NDSampler.resonance.RMatrixLimited.Parameters_RML_RRR import RMatrixLimited

# StoreRMatrixLimited = RMatrixLimited()
# StoreRMatrixLimited.from_endftk(range = RANGE)

# Test assignment
# StoreRMatrixLimited.ListSpinGroup[0].ResonanceParameters.ER[1] = -200

mf2mt151 = tape.MAT(tape.material_numbers[0]).MF(2).MT(151).parse()
original_isotope = mf2mt151.isotopes[0]
resonance_ranges = original_isotope.resonance_ranges.to_list()

NER = 0

MATMF32MT151 = tape.MAT(tape.material_numbers[0]).MF(32).MT(151).parse()

# Create new resonance range
new_range = ENDFtk.MF2.MT151.ResonanceRange( el = resonance_ranges[NER].EL, 
                                             eh = resonance_ranges[NER].EH, 
                                             naps = resonance_ranges[NER].NAPS, 
                                             parameters = RMatrixLimited().from_endftk(mf2_range     = RANGE, 
                                                                                       mf32_range    = MATMF32MT151.isotopes[0].resonance_ranges[0], 
                                                                                       force_reduced = True ).reconstruct())

resonance_ranges[0] = new_range

# Create new isotope with updated resonance ranges
new_isotope = ENDFtk.MF2.MT151.Isotope( zai=original_isotope.ZAI, abn=original_isotope.ABN, lfw=original_isotope.LFW, ranges=resonance_ranges)

# Create new section with the updated isotope
new_section = ENDFtk.MF2.MT151.Section( zaid=mf2mt151.ZA, awr=mf2mt151.AWR, isotopes=[new_isotope])

# Replace the existing section in the tape
mat_num = tape.material_numbers[0]
tape.MAT(mat_num).MF(2).insert_or_replace(new_section)
tape.to_file(f'reduced_tape.endf')

# Test MF32 covariance matrix

In [5]:
import NDSampler
from NDSampler.resonance.RMatrixLimited.Parameters_RML_RRR import RMatrixLimited


len(RMatrixLimited().from_endftk(mf2_range = RANGE, mf32_range = MATMF32MT151.isotopes[0].resonance_ranges[0]).getListStandardDeviation()[1])

3598

In [6]:
MATMF32MT151 = tape.MAT(tape.material_numbers[0]).MF(32).MT(151).parse()
RANGE32 = MATMF32MT151.isotopes[0].resonance_ranges[0]
print("DAP = ", RANGE32.parameters.scattering_radius_uncertainty)

RANGE.parameters.spin_groups[0].AJ

DAP =  None


-1.0

In [7]:
std_devs = []



for spingroup in RANGE32.parameters.uncertainties.spin_groups.to_list():
    for iER, DER in enumerate(spingroup.parameters.DER[:]):
        std_devs.append(DER)
        for iCH in range(spingroup.NCH):
            std_devs.append(spingroup.parameters.DGAM[iER][iCH])

print("standard deviations:", len(std_devs))


standard deviations: 3598


In [8]:
total_resonances = sum( ((sgroup.NCH + 1 ) * sgroup.NRSA) for sgroup in RANGE32.parameters.uncertainties.spin_groups.to_list())

print("NNN", RANGE32.parameters.correlation_matrix.NNN)
print("NNN", total_resonances)

NNN 3598
NNN 3598


In [9]:
import ENDFtk
from ENDFtk.tree import Tape
import numpy as np
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt

import NDSampler
from NDSampler.resonance.RMatrixLimited.Parameters_RML_RRR import SpinGroup
from NDSampler.resonance.RMatrixLimited.Uncertainty_RML_RRR import Uncertainty_RML_RRR

tape = Tape.from_file('/home/sole-pie01/ndlib/endfb8-neutron/n-029_Cu_063.endf')# OK
MATMF2MT151 = tape.MAT(tape.material_numbers[0]).MF(2).MT(151).parse()
MATMF32MT151 = tape.MAT(tape.material_numbers[0]).MF(32).MT(151).parse()
isotopeIndex = 0 # Always zero
rangeIndex = 0 # User defined
latex_column_names = {
    'energy': r'$E$',
    'Cu64 + photon [inclusive] width': r'$\Gamma_\gamma$',
    'n + Cu63 width': r'$\Gamma_n$',
    'n + Cu63 width_2': r'$\Gamma_n2$'
}
object = Uncertainty_RML_RRR(MATMF2MT151.isotopes[isotopeIndex].resonance_ranges[rangeIndex], 
                                  MATMF32MT151.isotopes[isotopeIndex].resonance_ranges[rangeIndex],
                                  rangeIndex)

variance_vector = np.diag(object.covariance_matrix)

# Example: retrieve parameters for spin group 0 (J=0) and parameter index 0 (P=0)
J, P = 0, 0
param_list = []
for i, (j, r, p) in enumerate(object.index_mapping):
    if j == J and p == P:
        if P == 0:
            # Resonance energy
            param_list.append(object.rml_data.ListSpinGroup[j].ResonanceParameters[r].ER[0])
        else:
            # Channel width
            param_list.append(object.rml_data.ListSpinGroup[j].ResonanceParameters[r].GAM[r][P - 1])

print(param_list)

object.index_mapping

def plot_relative_uncertainty(spin_group: SpinGroup, variance_vector: np.ndarray, param_type: int):
    """
    Plots the relative uncertainty vs resonance number.
    param_type = 0 -> resonance energies
    param_type = i > 0 -> width of the i-th channel
    """
    # Gather data
    values = []
    for idx, resonance in enumerate(spin_group.ResonanceParameters):
        if param_type == 0:
            nominal = resonance.ER[0] if resonance.ER else 0.0
        else:
            ch_idx = param_type - 1
            nominal = resonance.GAM[idx][ch_idx] if resonance.GAM and idx < len(resonance.GAM) else 0.0
        values.append(nominal)

    values = np.array(values)
    # Extract the relevant variances
    # (Assuming a simple 1D slice from variance_vector per resonance.)
    # If each resonance has 1 parameter for energies or for i-th width, then:
    uncertainties = np.sqrt(variance_vector[:len(values)])
    # Compute relative (percent) uncertainty
    rel_unc = np.where(values != 0, (uncertainties / np.abs(values))*100.0, 0.0)

    # Plot
    plt.figure()
    plt.bar(range(len(values)), rel_unc, color='blue', alpha=0.7)
    plt.xlabel("Resonance Number")
    plt.ylabel("Relative Uncertainty (%)")
    plt.title("SpinGroup Relative Uncertainty" if param_type == 0 else f"Channel {param_type} Width Relative Unc.")
    plt.show()
    
plot_relative_uncertainty(object.rml_data.spin_group, object.rml_data.variance_vector, 0)

Time for RMatrixLimited.from_endftk: 0.0146 seconds


AttributeError: 'SpinGroup' object has no attribute 'ResonanceParameters'

[-37843.81, -266.7135, 650.7223, 2047.718, 3308.797, 3501.424, 4395.985, 4434.962, 4854.599, 5398.48, 6838.426, 7563.271, 8626.676, 8647.264, 9759.164, 9919.925, 10308.96, 13702.23, 15090.58, 15805.69, 17863.64, 18340.37, 20389.29, 20444.08, 20919.15, 21015.53, 21522.22, 23625.45, 25748.12, 29529.59, 29930.89, 30472.25, 31395.49, 32205.6, 35017.79, 36149.88, 37963.58, 39109.76, 42541.18, 44702.45, 44761.87, 45823.63, 52921.78, 53744.86, 55342.94, 57680.21, 59132.02, 59440.19, 59935.39, 64506.39, 64989.91, 66284.36, 68937.99, 69653.9, 72956.95, 73686.63, 74595.01, 79148.71, 81353.74, 81842.44, 85924.26, 87503.58, 87569.87, 89539.66, 90265.56, 94687.84, 95301.52, 96887.9, 97202.58, 98400.58, 98536.2, 98822.81, 101596.0, 102675.1, 104523.2, 105286.3, 107127.5, 107343.4, 107735.9, 108730.4, 109612.3, 114829.0, 115225.8, 116979.8, 117251.0, 118247.4, 120059.2, 121140.6, 124244.2, 124518.4, 125351.0, 125614.6, 125739.0, 126708.2, 126911.8, 129774.4, 131352.3, 132633.9, 133997.0, 135176.4, 13

In [29]:
import ENDFtk
from ENDFtk.Tree import Tape
import numpy as np
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt

import NDSampler
from NDSampler.resonance.RMatrixLimited.Parameters import SpinGroup
from NDSampler.resonance.RMatrixLimited.Uncertainty_RML_RRR import Uncertainty_RML_RRR

# def plot_uncertainty(endf_tape : Tape):
    
    
#     tape = Tape.from_file('/home/sole-pie01/ndlib/endfb8-neutron/n-029_Cu_063.endf')# OK

#     MATMF2MT151 = tape.MAT(tape.material_numbers[0]).MF(2).MT(151).parse()
#     MATMF32MT151 = tape.MAT(tape.material_numbers[0]).MF(32).MT(151).parse()

#     isotopeIndex = 0 # Always zero
#     rangeIndex = 0 # User defined
    
#     latex_column_names = {
#         'energy': r'$E$',
#         'Cu64 + photon [inclusive] width': r'$\Gamma_\gamma$',
#         'n + Cu63 width': r'$\Gamma_n$',
#         'n + Cu63 width_2': r'$\Gamma_n2$'
#     }
    
#     object = Uncertainty_RML_RRR(MATMF2MT151.isotopes[isotopeIndex].resonance_ranges[rangeIndex], 
#                                       MATMF32MT151.isotopes[isotopeIndex].resonance_ranges[rangeIndex], 
#                                       rangeIndex)

#     # Assuming spin_group_data and parameterCov are already defined
#     plot_all_spin_groups(object.rml_data.ListSpinGroup, object.covariance_matrix, latex_column_names, show_labels=False)
    
def plot_relative_uncertainty(spin_group: SpinGroup, variance_vector: np.ndarray, param_type: int):
    """
    Plots the relative uncertainty vs resonance number.
    param_type = 0 -> resonance energies
    param_type = i > 0 -> width of the i-th channel
    """
    # Gather data
    values = []
    for idx, resonance in enumerate(spin_group.ResonanceParameters):
        if param_type == 0:
            nominal = resonance.ER[0] if resonance.ER else 0.0
        else:
            ch_idx = param_type - 1
            nominal = resonance.GAM[idx][ch_idx] if resonance.GAM and idx < len(resonance.GAM) else 0.0
        values.append(nominal)

    values = np.array(values)
    # Extract the relevant variances
    # (Assuming a simple 1D slice from variance_vector per resonance.)
    # If each resonance has 1 parameter for energies or for i-th width, then:
    uncertainties = np.sqrt(variance_vector[:len(values)])
    # Compute relative (percent) uncertainty
    rel_unc = np.where(values != 0, (uncertainties / np.abs(values))*100.0, 0.0)

    # Plot
    plt.figure()
    plt.bar(range(len(values)), rel_unc, color='blue', alpha=0.7)
    plt.xlabel("Resonance Number")
    plt.ylabel("Relative Uncertainty (%)")
    plt.title("SpinGroup Relative Uncertainty" if param_type == 0 else f"Channel {param_type} Width Relative Unc.")
    plt.show()
    
# def plot_all_spin_groups(object: Uncertainty_RML_RRR, latex_column_names=None, show_labels=True):
#     num_groups = len(object.rml_data.ListSpinGroup)
    
#     object.rml_data.ListSpinGroup[0].ResonanceChannels.spin
    
#     # Find the maximum number of parameters across all groups
#     max_params = max([group_data['data'].shape[0] for group_data in spin_group_data.values()])
#     num_parameters = 4  # We know each group has 3 parameters

#     # Create a grid with an extra row for titles and an extra column for row names
#     fig = plt.figure(figsize=(15, 2 * (num_groups + 1)))  # Adjust the 3 to control the overall height
#     # The first element in width_ratios=[1.5] + [4] * num_parameters is use to tune the width of the first column
#     gs = GridSpec(num_groups + 1, num_parameters + 1, figure=fig, height_ratios=[0.000001] + [1] * num_groups, width_ratios=[2.5] + [4] * num_parameters)
    
#     # Set up column titles
#     first_group = next(iter(spin_group_data.values()))['data']
#     param_names = first_group.columns
#     for j, param_name in enumerate(param_names):
#         ax = fig.add_subplot(gs[0, j + 1])
#         ax.set_title(latex_column_names.get(param_name, param_name))
#         ax.axis('off')

#     # Set up row names and plots for each spin group
#     global_param_index = 0
#     for i, (group_label, group_data) in enumerate(spin_group_data.items()):
#         df = group_data['data']
#         # print("row number ", i)
#         # print(df)
#         # Extract the diagonal of the covariance matrix
#         start_index = sum([spin_group_data[key]['data'].shape[0] * spin_group_data[key]['data'].shape[1] for key in spin_group_data if key < group_label])
#         end_index = start_index + (df.shape[1] * df.shape[0])
#         variances = np.array(variance_vector[start_index:end_index]).reshape((df.shape[0], df.shape[1]))

#         # Set row names
#         spin = group_data['spin']
#         parity = group_data['parity']
#         parity_symbol = "+" if int(parity) > 0 else "-"
#         row_name = f"$J^{{\Pi}} = {spin}^{parity_symbol}$"
#         row_ax = fig.add_subplot(gs[i + 1, 0])
#         row_ax.text(0.5, 0.5, row_name, ha='center', va='center', transform=row_ax.transAxes, fontsize=14)
#         row_ax.axis('off')
        
#         # Plot parameters
#         group_axes = [fig.add_subplot(gs[i + 1, j + 1]) for j in range(df.shape[1])]
#         for j in range(df.shape[1]):
#             show_y = (j == 1)  # Show y-axis label only for the second column
#             show_x = (i == num_groups - 1)  # Show x-axis label only for the last row
#             global_param_index = plot_parameters_with_relative_uncertainty(group_axes, df, variances, max_params, global_param_index, show_y, show_x)
    
#     plt.tight_layout(pad=2.0, h_pad=2.0, w_pad=2.0)
#     plt.show()

2